In [1]:
import torch
from torch import nn
import lightnet as ln
from torchinfo import summary
from model import model_builder
import functools

/home/gustavo/workstation/depth_estimation/codes/rgbd-yolov2/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# model = ln.models.Darknet19(1000)
# model.load('weights/darknet19_448.weights')

# # Save as PyTorch weight file (Not strictly necessary, but it is faster than darknet weight files)
# model.save('weights/darknet19_448.pt')

# # Converting Darknet19 weights to Yolo (This is the same as the darknet19_448.conv.23.weights from darknet)
# model.save('weights/yolo-pretrained_darknet.pt', remap=ln.models.YoloV2.remap_darknet19)

# # Load yolo weights (Requires `strict=False`, because not all layers have weights in this file)
# detection_model = ln.models.YoloV2(3)
# detection_model.load('weights/yolo-pretrained_darknet.pt', strict=False)

## Depth Only Network

In [3]:
rgb_model = model_builder(num_classes=3, model_type="rgb")
summary(model=rgb_model, 
        input_size=(32, 3, 416, 416), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

/home/gustavo/workstation/depth_estimation/codes/rgbd-yolov2/.venv/lib/python3.10/site-packages/lightnet/network/module/_lightnet.py:293: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental 

Layer (type (var_name))                       Input Shape          Output Shape         Param #              Trainable
YoloV2 (YoloV2)                               [32, 3, 416, 416]    [32, 40, 13, 13]     --                   True
├─FeatureExtractor (backbone)                 [32, 3, 416, 416]    [32, 1024, 13, 13]   --                   True
│    └─Sequential (module)                    [32, 3, 416, 416]    [32, 1024, 13, 13]   --                   True
│    │    └─Conv2dBatchAct (1_convbatch)      [32, 3, 416, 416]    [32, 32, 416, 416]   928                  True
│    │    └─MaxPool2d (2_max)                 [32, 32, 416, 416]   [32, 32, 208, 208]   --                   --
│    │    └─Conv2dBatchAct (3_convbatch)      [32, 32, 208, 208]   [32, 64, 208, 208]   18,560               True
│    │    └─MaxPool2d (4_max)                 [32, 64, 208, 208]   [32, 64, 104, 104]   --                   --
│    │    └─Conv2dBatchAct (5_convbatch)      [32, 64, 104, 104]   [32, 128, 104, 104] 

In [4]:
depth_model = model_builder(num_classes=3, model_type="depth")

# Print a summary using torchinfo (uncomment for actual output)
summary(model=depth_model, 
        input_size=(32, 1, 416, 416), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
) 

Modules not matching, performing partial update


Replacing first layer with new conv: Conv2dBatchAct(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), LeakyReLU(negative_slope=0.1, inplace=True))


Layer (type (var_name))                       Input Shape          Output Shape         Param #              Trainable
YoloV2 (YoloV2)                               [32, 1, 416, 416]    [32, 40, 13, 13]     --                   True
├─FeatureExtractor (backbone)                 [32, 1, 416, 416]    [32, 1024, 13, 13]   --                   True
│    └─Sequential (module)                    [32, 1, 416, 416]    [32, 1024, 13, 13]   --                   True
│    │    └─Conv2dBatchAct (1_convbatch)      [32, 1, 416, 416]    [32, 32, 416, 416]   352                  True
│    │    └─MaxPool2d (2_max)                 [32, 32, 416, 416]   [32, 32, 208, 208]   --                   --
│    │    └─Conv2dBatchAct (3_convbatch)      [32, 32, 208, 208]   [32, 64, 208, 208]   18,560               True
│    │    └─MaxPool2d (4_max)                 [32, 64, 208, 208]   [32, 64, 104, 104]   --                   --
│    │    └─Conv2dBatchAct (5_convbatch)      [32, 64, 104, 104]   [32, 128, 104, 104] 

In [5]:
fusion_model = model_builder(num_classes=3, model_type="rgbd", fuse_layer=15)

In [6]:
input_tensor = torch.randn(1, 4, 416, 416)
output_tensor = fusion_model(input_tensor)
print(output_tensor.shape)

torch.Size([1, 40, 13, 13])


In [7]:
torch.save(rgb_model.state_dict(), "models/rgb_state_dict.pth")

In [8]:
rgb_state_dict = torch.load("models/rgb_state_dict.pth")

/tmp/ipykernel_58122/3272559930.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rgb_state_dict = torch.load("models/rgb_state_dict.pth")


In [9]:
rgb_state_dict

OrderedDict([('backbone.module.1_convbatch.layers.0.weight',
              tensor([[[[-5.7406e-02,  1.1122e-02,  2.5896e-02],
                        [-1.2880e-01,  2.1973e-02,  8.8995e-02],
                        [-1.9058e-02,  3.6254e-02,  1.9450e-02]],
              
                       [[-5.6026e-02,  3.3901e-02,  6.2830e-02],
                        [-1.3696e-01,  2.4380e-02,  9.9916e-02],
                        [-5.4075e-02,  1.7731e-02,  2.1061e-02]],
              
                       [[-1.8691e-02,  1.0901e-02,  2.1609e-02],
                        [-9.0229e-02,  9.4340e-04,  5.5368e-02],
                        [ 6.2052e-04,  4.5239e-03, -2.8572e-03]]],
              
              
                      [[[ 2.7070e-02,  4.7068e-02,  5.6571e-02],
                        [ 2.6374e-02, -5.3922e-02, -5.2401e-02],
                        [ 2.9353e-02, -9.2439e-02, -5.5904e-02]],
              
                       [[ 3.7628e-02,  4.5574e-02,  6.0078e-02],
              

## Remapping layers

In [10]:
remap_rgbd_15_fusion_model = {
    'backbone.module.1_convbatch.layers.0.weight': 'layers.0.1_convbatch_regular.layers.0.weight',
    'backbone.module.1_convbatch.layers.1.weight': 'layers.0.1_convbatch_regular.layers.1.weight',
    'backbone.module.1_convbatch.layers.1.bias': 'layers.0.1_convbatch_regular.layers.1.bias',
    'backbone.module.1_convbatch.layers.1.running_mean': 'layers.0.1_convbatch_regular.layers.1.running_mean',
    'backbone.module.1_convbatch.layers.1.running_var': 'layers.0.1_convbatch_regular.layers.1.running_var',
    'backbone.module.1_convbatch.layers.1.num_batches_tracked': 'layers.0.1_convbatch_regular.layers.1.num_batches_tracked',
    'backbone.module.3_convbatch.layers.0.weight': 'layers.1.regular.3_convbatch.layers.0.weight',
    'backbone.module.3_convbatch.layers.1.weight': 'layers.1.regular.3_convbatch.layers.1.weight',
    'backbone.module.3_convbatch.layers.1.bias': 'layers.1.regular.3_convbatch.layers.1.bias',
    'backbone.module.3_convbatch.layers.1.running_mean': 'layers.1.regular.3_convbatch.layers.1.running_mean',
    'backbone.module.3_convbatch.layers.1.running_var': 'layers.1.regular.3_convbatch.layers.1.running_var',
    'backbone.module.3_convbatch.layers.1.num_batches_tracked': 'layers.1.regular.3_convbatch.layers.1.num_batches_tracked',
    'backbone.module.5_convbatch.layers.0.weight': 'layers.1.regular.5_convbatch.layers.0.weight',
    'backbone.module.5_convbatch.layers.1.weight': 'layers.1.regular.5_convbatch.layers.1.weight',
    'backbone.module.5_convbatch.layers.1.bias': 'layers.1.regular.5_convbatch.layers.1.bias',
    'backbone.module.5_convbatch.layers.1.running_mean': 'layers.1.regular.5_convbatch.layers.1.running_mean',
    'backbone.module.5_convbatch.layers.1.running_var': 'layers.1.regular.5_convbatch.layers.1.running_var',
    'backbone.module.5_convbatch.layers.1.num_batches_tracked': 'layers.1.regular.5_convbatch.layers.1.num_batches_tracked',
    'backbone.module.6_convbatch.layers.0.weight': 'layers.1.regular.6_convbatch.layers.0.weight',
    'backbone.module.6_convbatch.layers.1.weight': 'layers.1.regular.6_convbatch.layers.1.weight',
    'backbone.module.6_convbatch.layers.1.bias': 'layers.1.regular.6_convbatch.layers.1.bias',
    'backbone.module.6_convbatch.layers.1.running_mean': 'layers.1.regular.6_convbatch.layers.1.running_mean',
    'backbone.module.6_convbatch.layers.1.running_var': 'layers.1.regular.6_convbatch.layers.1.running_var',
    'backbone.module.6_convbatch.layers.1.num_batches_tracked': 'layers.1.regular.6_convbatch.layers.1.num_batches_tracked',
    'backbone.module.7_convbatch.layers.0.weight' : 'layers.1.regular.7_convbatch.layers.0.weight',
    'backbone.module.7_convbatch.layers.1.weight' : 'layers.1.regular.7_convbatch.layers.1.weight',
    'backbone.module.7_convbatch.layers.1.bias' : 'layers.1.regular.7_convbatch.layers.1.bias',
    'backbone.module.7_convbatch.layers.1.running_mean': 'layers.1.regular.7_convbatch.layers.1.running_mean',
    'backbone.module.7_convbatch.layers.1.running_var':  'layers.1.regular.7_convbatch.layers.1.running_var',
    'backbone.module.7_convbatch.layers.1.num_batches_tracked':  'layers.1.regular.7_convbatch.layers.1.num_batches_tracked',
    'backbone.module.9_convbatch.layers.0.weight': 'layers.1.regular.9_convbatch.layers.0.weight',
    'backbone.module.9_convbatch.layers.1.weight': 'layers.1.regular.9_convbatch.layers.1.weight',
    'backbone.module.9_convbatch.layers.1.bias': 'layers.1.regular.9_convbatch.layers.1.bias',
    'backbone.module.9_convbatch.layers.1.running_mean': 'layers.1.regular.9_convbatch.layers.1.running_mean',
    'backbone.module.9_convbatch.layers.1.running_var': 'layers.1.regular.9_convbatch.layers.1.running_var',
    'backbone.module.9_convbatch.layers.1.num_batches_tracked': 'layers.1.regular.9_convbatch.layers.1.num_batches_tracked',
    'backbone.module.10_convbatch.layers.0.weight': 'layers.1.regular.10_convbatch.layers.0.weight',
    'backbone.module.10_convbatch.layers.1.weight': 'layers.1.regular.10_convbatch.layers.1.weight',
    'backbone.module.10_convbatch.layers.1.bias': 'layers.1.regular.10_convbatch.layers.1.bias',
    'backbone.module.10_convbatch.layers.1.running_mean': 'layers.1.regular.10_convbatch.layers.1.running_mean',
    'backbone.module.10_convbatch.layers.1.running_var': 'layers.1.regular.10_convbatch.layers.1.running_var',
    'backbone.module.10_convbatch.layers.1.num_batches_tracked': 'layers.1.regular.10_convbatch.layers.1.num_batches_tracked',
    'backbone.module.11_convbatch.layers.0.weight': 'layers.1.regular.11_convbatch.layers.0.weight',
    'backbone.module.11_convbatch.layers.1.weight': 'layers.1.regular.11_convbatch.layers.1.weight',
    'backbone.module.11_convbatch.layers.1.bias': 'layers.1.regular.11_convbatch.layers.1.bias',
    'backbone.module.11_convbatch.layers.1.running_mean': 'layers.1.regular.11_convbatch.layers.1.running_mean',
    'backbone.module.11_convbatch.layers.1.running_var': 'layers.1.regular.11_convbatch.layers.1.running_var',
    'backbone.module.11_convbatch.layers.1.num_batches_tracked': 'layers.1.regular.11_convbatch.layers.1.num_batches_tracked',
    'backbone.module.13_convbatch.layers.0.weight': 'layers.1.regular.13_convbatch.layers.0.weight',
    'backbone.module.13_convbatch.layers.1.weight': 'layers.1.regular.13_convbatch.layers.1.weight',
    'backbone.module.13_convbatch.layers.1.bias': 'layers.1.regular.13_convbatch.layers.1.bias',
    'backbone.module.13_convbatch.layers.1.running_mean': 'layers.1.regular.13_convbatch.layers.1.running_mean',
    'backbone.module.13_convbatch.layers.1.running_var': 'layers.1.regular.13_convbatch.layers.1.running_var',
    'backbone.module.13_convbatch.layers.1.num_batches_tracked': 'layers.1.regular.13_convbatch.layers.1.num_batches_tracked',
    'backbone.module.14_convbatch.layers.0.weight': 'layers.1.regular.14_convbatch.layers.0.weight',
    'backbone.module.14_convbatch.layers.1.weight': 'layers.1.regular.14_convbatch.layers.1.weight',
    'backbone.module.14_convbatch.layers.1.bias': 'layers.1.regular.14_convbatch.layers.1.bias',
    'backbone.module.14_convbatch.layers.1.running_mean': 'layers.1.regular.14_convbatch.layers.1.running_mean',
    'backbone.module.14_convbatch.layers.1.running_var': 'layers.1.regular.14_convbatch.layers.1.running_var',
    'backbone.module.14_convbatch.layers.1.num_batches_tracked': 'layers.1.regular.14_convbatch.layers.1.num_batches_tracked',
    'backbone.module.15_convbatch.layers.0.weight': 'layers.1.regular.15_convbatch.layers.0.weight',
    'backbone.module.15_convbatch.layers.1.weight': 'layers.1.regular.15_convbatch.layers.1.weight',
    'backbone.module.15_convbatch.layers.1.bias': 'layers.1.regular.15_convbatch.layers.1.bias',
    'backbone.module.15_convbatch.layers.1.running_mean': 'layers.1.regular.15_convbatch.layers.1.running_mean',
    'backbone.module.15_convbatch.layers.1.running_var': 'layers.1.regular.15_convbatch.layers.1.running_var',
    'backbone.module.15_convbatch.layers.1.num_batches_tracked': 'layers.1.regular.15_convbatch.layers.1.num_batches_tracked',
    'backbone.module.16_convbatch.layers.0.weight': 'layers.1.combined.16_convbatch.layers.0.weight',
    'backbone.module.16_convbatch.layers.1.weight': 'layers.1.combined.16_convbatch.layers.1.weight',
    'backbone.module.16_convbatch.layers.1.bias': 'layers.1.combined.16_convbatch.layers.1.bias',
    'backbone.module.16_convbatch.layers.1.running_mean': 'layers.1.combined.16_convbatch.layers.1.running_mean',
    'backbone.module.16_convbatch.layers.1.running_var': 'layers.1.combined.16_convbatch.layers.1.running_var',
    'backbone.module.16_convbatch.layers.1.num_batches_tracked': 'layers.1.combined.16_convbatch.layers.1.num_batches_tracked',
    'backbone.module.17_convbatch.layers.0.weight': 'layers.1.combined.17_convbatch.layers.0.weight',
    'backbone.module.17_convbatch.layers.1.weight': 'layers.1.combined.17_convbatch.layers.1.weight',
    'backbone.module.17_convbatch.layers.1.bias': 'layers.1.combined.17_convbatch.layers.1.bias',
    'backbone.module.17_convbatch.layers.1.running_mean': 'layers.1.combined.17_convbatch.layers.1.running_mean',
    'backbone.module.17_convbatch.layers.1.running_var': 'layers.1.combined.17_convbatch.layers.1.running_var',
    'backbone.module.17_convbatch.layers.1.num_batches_tracked': 'layers.1.combined.17_convbatch.layers.1.num_batches_tracked',
    'backbone.module.19_convbatch.layers.0.weight': 'layers.2.combined.19_convbatch.layers.0.weight',
    'backbone.module.19_convbatch.layers.1.weight': 'layers.2.combined.19_convbatch.layers.1.weight',
    'backbone.module.19_convbatch.layers.1.bias': 'layers.2.combined.19_convbatch.layers.1.bias',
    'backbone.module.19_convbatch.layers.1.running_mean': 'layers.2.combined.19_convbatch.layers.1.running_mean',
    'backbone.module.19_convbatch.layers.1.running_var': 'layers.2.combined.19_convbatch.layers.1.running_var',
    'backbone.module.19_convbatch.layers.1.num_batches_tracked': 'layers.2.combined.19_convbatch.layers.1.num_batches_tracked',
    'backbone.module.20_convbatch.layers.0.weight': 'layers.2.combined.20_convbatch.layers.0.weight',
    'backbone.module.20_convbatch.layers.1.weight': 'layers.2.combined.20_convbatch.layers.1.weight',
    'backbone.module.20_convbatch.layers.1.bias': 'layers.2.combined.20_convbatch.layers.1.bias',
    'backbone.module.20_convbatch.layers.1.running_mean': 'layers.2.combined.20_convbatch.layers.1.running_mean',
    'backbone.module.20_convbatch.layers.1.running_var': 'layers.2.combined.20_convbatch.layers.1.running_var',
    'backbone.module.20_convbatch.layers.1.num_batches_tracked': 'layers.2.combined.20_convbatch.layers.1.num_batches_tracked',
    'backbone.module.21_convbatch.layers.0.weight': 'layers.2.combined.21_convbatch.layers.0.weight',
    'backbone.module.21_convbatch.layers.1.weight': 'layers.2.combined.21_convbatch.layers.1.weight',
    'backbone.module.21_convbatch.layers.1.bias': 'layers.2.combined.21_convbatch.layers.1.bias',
    'backbone.module.21_convbatch.layers.1.running_mean': 'layers.2.combined.21_convbatch.layers.1.running_mean',
    'backbone.module.21_convbatch.layers.1.running_var': 'layers.2.combined.21_convbatch.layers.1.running_var',
    'backbone.module.21_convbatch.layers.1.num_batches_tracked': 'layers.2.combined.21_convbatch.layers.1.num_batches_tracked',
    'backbone.module.22_convbatch.layers.0.weight': 'layers.2.combined.22_convbatch.layers.0.weight',
    'backbone.module.22_convbatch.layers.1.weight': 'layers.2.combined.22_convbatch.layers.1.weight',
    'backbone.module.22_convbatch.layers.1.bias': 'layers.2.combined.22_convbatch.layers.1.bias',
    'backbone.module.22_convbatch.layers.1.running_mean': 'layers.2.combined.22_convbatch.layers.1.running_mean',
    'backbone.module.22_convbatch.layers.1.running_var': 'layers.2.combined.22_convbatch.layers.1.running_var',
    'backbone.module.22_convbatch.layers.1.num_batches_tracked': 'layers.2.combined.22_convbatch.layers.1.num_batches_tracked',
    'backbone.module.23_convbatch.layers.0.weight': 'layers.2.combined.23_convbatch.layers.0.weight',
    'backbone.module.23_convbatch.layers.1.weight': 'layers.2.combined.23_convbatch.layers.1.weight',
    'backbone.module.23_convbatch.layers.1.bias': 'layers.2.combined.23_convbatch.layers.1.bias',
    'backbone.module.23_convbatch.layers.1.running_mean': 'layers.2.combined.23_convbatch.layers.1.running_mean',
    'backbone.module.23_convbatch.layers.1.running_var': 'layers.2.combined.23_convbatch.layers.1.running_var',
    'backbone.module.23_convbatch.layers.1.num_batches_tracked': 'layers.2.combined.23_convbatch.layers.1.num_batches_tracked'
}

In [11]:
def rename_state_dict(state_dict, layer_mapping):
    renamed_state_dict = {}
    for src_layer, tgt_layer in layer_mapping.items():
        if src_layer in state_dict:
            renamed_state_dict[tgt_layer] = state_dict[src_layer]
        else:
            print(f"Warning: {src_layer} not found in the source state_dict.")
    return renamed_state_dict

In [12]:
renamed_rgb_state_dict = rename_state_dict(rgb_state_dict, remap_rgbd_15_fusion_model)

In [13]:
renamed_rgb_state_dict

{'layers.0.1_convbatch_regular.layers.0.weight': tensor([[[[-5.7406e-02,  1.1122e-02,  2.5896e-02],
           [-1.2880e-01,  2.1973e-02,  8.8995e-02],
           [-1.9058e-02,  3.6254e-02,  1.9450e-02]],
 
          [[-5.6026e-02,  3.3901e-02,  6.2830e-02],
           [-1.3696e-01,  2.4380e-02,  9.9916e-02],
           [-5.4075e-02,  1.7731e-02,  2.1061e-02]],
 
          [[-1.8691e-02,  1.0901e-02,  2.1609e-02],
           [-9.0229e-02,  9.4340e-04,  5.5368e-02],
           [ 6.2052e-04,  4.5239e-03, -2.8572e-03]]],
 
 
         [[[ 2.7070e-02,  4.7068e-02,  5.6571e-02],
           [ 2.6374e-02, -5.3922e-02, -5.2401e-02],
           [ 2.9353e-02, -9.2439e-02, -5.5904e-02]],
 
          [[ 3.7628e-02,  4.5574e-02,  6.0078e-02],
           [ 3.0111e-02, -7.7550e-02, -7.3636e-02],
           [ 3.4858e-02, -9.9474e-02, -5.6496e-02]],
 
          [[-5.3021e-02, -5.4680e-02, -3.0603e-02],
           [-5.2830e-02, -8.0152e-02, -5.9254e-02],
           [-1.2189e-02, -5.7931e-02, -1.8048e-02]

In [14]:
target_fusion_model = model_builder(num_classes=3, model_type="rgbd", fuse_layer=15)

In [15]:
target_fusion_model.load_state_dict(renamed_rgb_state_dict, strict=False)

_IncompatibleKeys(missing_keys=['layers.0.1_convbatch_fusion.layers.0.weight', 'layers.0.1_convbatch_fusion.layers.1.weight', 'layers.0.1_convbatch_fusion.layers.1.bias', 'layers.0.1_convbatch_fusion.layers.1.running_mean', 'layers.0.1_convbatch_fusion.layers.1.running_var', 'layers.1.fusion.3_convbatch.layers.0.weight', 'layers.1.fusion.3_convbatch.layers.1.weight', 'layers.1.fusion.3_convbatch.layers.1.bias', 'layers.1.fusion.3_convbatch.layers.1.running_mean', 'layers.1.fusion.3_convbatch.layers.1.running_var', 'layers.1.fusion.5_convbatch.layers.0.weight', 'layers.1.fusion.5_convbatch.layers.1.weight', 'layers.1.fusion.5_convbatch.layers.1.bias', 'layers.1.fusion.5_convbatch.layers.1.running_mean', 'layers.1.fusion.5_convbatch.layers.1.running_var', 'layers.1.fusion.6_convbatch.layers.0.weight', 'layers.1.fusion.6_convbatch.layers.1.weight', 'layers.1.fusion.6_convbatch.layers.1.bias', 'layers.1.fusion.6_convbatch.layers.1.running_mean', 'layers.1.fusion.6_convbatch.layers.1.runnin

In [20]:
fusion_weight = target_fusion_model.state_dict()['layers.1.regular.11_convbatch.layers.1.weight'].cpu()
rgb_weight = rgb_model.state_dict()['backbone.module.11_convbatch.layers.1.weight'].cpu()

In [21]:
torch.equal(fusion_weight, rgb_weight)

True